In [1]:
USE_FAST = True

In [2]:
import time
import numpy as np

np.random.seed(0)

if USE_FAST:
    from lib.fast.linear_algebra import Matrix
    from lib.fast.nn import Linear, ReLU, NN
    from lib.fast.processing import ColumnNormalizer
else:
    from lib.original.linear_algebra import Matrix
    from lib.original.nn import Linear, ReLU, NN
    from lib.original.processing import ColumnNormalizer
from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader
from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

In [3]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [4]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

((404, 13), (404, 1), (102, 13), (102, 1))

In [5]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

((404, 13), (102, 13))

In [6]:
def init_nn():
    return NN([
        Linear(13, 4),
        ReLU(),
        Linear(4, 1),
    ])

In [7]:
time_point = time.time()

data_loaders = [
    # BatchDataLoader(X_train, y_train),
    # StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]
optimizer_creators = [
    # lambda nn: SgdOptimizer(nn, 0.001),
    # lambda nn: SgdWithMomentumOptimizer(nn, 0.001, 0.9),
    lambda nn: AdaGradOptimizer(nn, 0.05),
    lambda nn: RmsPropOptimizer(nn, 0.01, 0.95),
    lambda nn: AdamOptimizer(nn, 0.01, 0.95, 0.95),
]

for data_loader in data_loaders:
    for optimizer_creator in optimizer_creators:
        nn = init_nn()
        optimizer = optimizer_creator(nn)
        print(f"gradient descent: {data_loader.__class__} | optimizer: {optimizer.__class__}")
        for i in range(2001):
            X_b, y_b = data_loader.get_batch()
            out = nn(X_b)
            loss = mean_squared_error(y_b, out)
            if i % 100 == 0:
                elapsed_time = int(time.time() - time_point)
                time_point = time.time()
                print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

            optimizer.step(loss)

        train_out = nn(X_train) 
        train_loss = mean_squared_error(y_train, train_out)
        test_out = nn(X_test) 
        test_loss = mean_squared_error(y_test, test_out)
        print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.AdaGradOptimizer'>
0 | 570.99 | 0s
100 | 188.97 | 14s
200 | 64.86 | 4s
300 | 63.16 | 14s
400 | 41.55 | 14s
500 | 55.41 | 4s
600 | 27.28 | 15s
700 | 29.82 | 16s
800 | 42.44 | 14s
900 | 15.94 | 4s
1000 | 18.59 | 15s
1100 | 28.32 | 18s
1200 | 39.23 | 4s
1300 | 42.60 | 21s
1400 | 28.50 | 18s
1500 | 16.40 | 4s
1600 | 11.44 | 16s
1700 | 14.21 | 16s
1800 | 12.42 | 4s
1900 | 18.77 | 18s
2000 | 45.53 | 18s
train loss: 20.85   test loss: 21.01
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.optimizers.RmsPropOptimizer'>
0 | 549.58 | 0s
100 | 96.48 | 23s
200 | 19.86 | 4s
300 | 16.44 | 19s
400 | 37.44 | 21s
500 | 22.77 | 5s
600 | 6.73 | 19s
700 | 12.32 | 18s
800 | 15.24 | 4s
900 | 42.81 | 19s
1000 | 5.42 | 19s
1100 | 7.29 | 5s
1200 | 18.53 | 17s
1300 | 18.91 | 20s
1400 | 14.87 | 5s
1500 | 7.27 | 17s
1600 | 24.21 | 20s
1700 | 8.31 | 21s
1800 | 8.33 | 4

In [8]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out], [v[0].data for v in y_test])][:10]

[(42.4, 20.7),
 (17.4, 39.8),
 (33.7, 17.8),
 (6.7, 19.6),
 (16.3, 14.9),
 (26.4, 22.0),
 (14.6, 48.8),
 (21.6, 25.0),
 (19.4, 48.5),
 (17.2, 23.9)]